# MACD(Moving Average Convergence / Divergence )
MACD（Moving Average Convergence and Divergence) 是Geral Appel 于1979年提出的，利用收盘价的短期（常用为12日）指数移动平均线与长期（常用为26日）指数移动平均线之间的聚合与分离状况，对买进、卖出时机作出研判的技术指标。

In [ ]:
##keep
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## 加载贵州茅台股价数据

## 取2020年1月1日开始的记录组成新的数据框

## 画出收盘价走势图

## 计算MACD和signal序列

## 画出MACD指标图

## 将macd和signal序列添加到原来的数据框

## 计算金叉与死叉

## 在收盘价趋势图上标识出金叉与死叉